## Direktni sustav upravljanja - kompenzacija poremećaja
## Simulacija

In [14]:
%matplotlib widget
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
import matplotlib.pyplot as plt
import lib.simulator as sim

In [15]:
# model procesa
def spremnik_topline(k_h, k_g, C_s):
    A = [-k_g/C_s]
    B = [k_h/C_s, k_g/C_s]
    C = [1]
    D = [0, 0]
    return A, B, C, D

In [16]:
# prijenosna funkcija kompenzatora
C_s = 1
k_g = 0.4
k_h = 0.8
Gcomp = sim.gain(-k_g/k_h)

In [17]:
# model cjelokupnog sustava

blocks = [
    sim.step(y0=0, y1=1, t_step=1),  #0
    sim.step(y0=0, y1=1, t_step=15), #1
    Gcomp,
    sim.suma('++'), #3
    sim.ss(*spremnik_topline(k_h=0.8, k_g=0.4, C_s=1),   # parametri spremnika
           [0.0])                                        # početni uvjeti (početna temperatura)
    ]

connections = [
    ((0, 0), (3, 0)),
    ((1, 0), (4, 1)),
    ((1, 0), (2, 0)),    
    ((2, 0), (3, 1)),    
    ((3, 0), (4, 0)),    
    
]

outputs = [0, 1, 3, 4]

t, y = sim.simulate(blocks, connections, outputs, t_final=30.0)

In [18]:
# vremenski odziv
fig = plt.figure()
fig.set_label('Odziv procesa')
ax1=plt.subplot(2, 1, 1)
plt.plot(t, y[:,1], t, y[:,3])
plt.grid()
plt.legend(['Vanjska temperatura', 'Temperatura spremnika'])
ax2=plt.subplot(2, 1, 2)
plt.plot(t, y[:,0], t, y[:,2])
plt.grid()
plt.legend(['Ulaz', 'Grijač'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …